# 📋 **Faculty Contact & Assignment Report Generator**

### This notebook automates the creation of two key deliverables from a semester course schedule:
*   A structured Excel report listing faculty teaching assignments by subject, section, and campus—along with their emails.
*   D2L-compatible text rosters (TXT files) for each course, listing faculty usernames for bulk upload into course templates.

##  Designed for use in Google Colab or Jupyter.

## 📦 Step 1: Import Required Libraries

In [133]:
pip install xlsxwriter

In [134]:
import pandas as pd
import numpy as np
import os
import xlsxwriter
from datetime import datetime
from pytz import timezone
from google.colab import files, drive
import csv

In [135]:
# Define known campus codes and their full names for context or future merging
campuses = ['PA','PE','PS','PC','PN', 'PF']
campuses_dict = {
    'PA': "Alpharetta",
    'PF': "Online",
    'PS': "Decatur",
    'PC': "Clarkston",
    'PE': "Newton",
    'PN': "Dunwoody"

}

## 📁 Step 2: Upload Semester Schedule CSV and Faculty Email List
*   If files are not already uploaded in the session, prompt for upload.

In [136]:
USE_GITHUB = True  # Set to False if uploading manually

if USE_GITHUB:
    schedule_url = "https://raw.githubusercontent.com/brownt47/faculty_lists/main/sample_data/CSV_Class_Schedule_By_Department_Sample.csv"
    email_url = "https://raw.githubusercontent.com/brownt47/faculty_lists/main/sample_data/Faculty_Emails_Example.csv"
    schedule_df = pd.read_csv(schedule_url)
    email_df = pd.read_csv(email_url)
else:
    from google.colab import files
    uploaded = files.upload()
    schedule_file = list(uploaded.keys())[0]
    schedule_df = pd.read_csv(schedule_file)


## 🔍 Step 3: Preview the Uploaded Data

In [137]:
schedule_df.head(3)

,TERM,COLL,CRN,DEPT,SAPR,XLST,PTRM,Subj_Crsnumb,CRSE_TITLE,SEQ_NUMB,...,SEC,SSBSECT_VOICE_AVAIL,WL_LIMIT,WL_ENRL,RSRV_IND,RESV_LIMIT,RESV_ENRL,NRESV_LIMIT,NRESV_ENRL,RESV_LIST
0,202505,PC,51722,12,NaN,NaN,7,MATH 0999,SUPPORT FOR COLLEGE ALGEBRA,16,...,A,Y,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,202505,PC,53481,12,NaN,NaN,7,MATH 0999,SUPPORT FOR COLLEGE ALGEBRA,20,...,A,Y,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,202505,PC,53358,12,NaN,NaN,7,MATH 1111,COLLEGE ALGEBRA,30,...,A,Y,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
email_df.head(3)

,Name,Email
0,"01 Lovelace, Ada",alovelace3@example.edu
1,"01 White, Walter",wwhite9@example.edu
2,"01 Doom, Victor",vdoom3@example.edu


## 🧹 Step 4: Clean, Prepare, and Standardize Data

In [139]:
# Clean up column names and instructor entries by stripping extra whitespace
schedule_df.columns = schedule_df.columns.str.strip()
schedule_df['INSTRUCTOR'] = schedule_df['INSTRUCTOR'].str.strip()

In [140]:
# Remove numeric prefix (e.g., "01 ") from instructor names. ex: "01 Lovelace, Ada"
# This prefix indicates instructor rank or load priority and is an artifact from the scheduling system
schedule_df['INSTRUCTOR'] = schedule_df['INSTRUCTOR'].str[3:]
email_df['Name'] = email_df['Name'].str[3:]

In [141]:
#Merge Instructor email addresses to schedule_df
merged_df = pd.merge(
    schedule_df,
    email_df.rename(columns={'Name': 'INSTRUCTOR'}),
    on='INSTRUCTOR',
    how='left'
)

In [142]:
# Remove unassigned/staffed sections
merged_df = merged_df[merged_df['INSTRUCTOR'].str.upper() != 'STAFF']

In [143]:
# Assign Campus Names by Campus Codes
merged_df['Campus_Name'] = merged_df['CAMP'].map(campuses_dict)

### Identify Support Linked Courses

> #### Change Subj_Crsnumb for classes that have a linked co-requisite support
> ##### Example: MATH 1111 and Math 0999 become Math 1111_0999


In [144]:
# Create HAS_SUPPORT flag:
# - Must be MATH 1111, 1001, or 1401
# - Must contain linked support "CRN" in the COMMENTS column

merged_df['HAS_SUPPORT'] = (
    merged_df['Subj_Crsnumb'].isin(['MATH 1111', 'MATH 1001', 'MATH 1401']) &
    merged_df['COMMENTS'].str.upper().str.contains('CRN', na=False)
).astype(int)

In [145]:
# If support is present, relabel main course as a paired course
merged_df.loc[
    (merged_df['HAS_SUPPORT'] == 1) & (merged_df['Subj_Crsnumb'] == 'MATH 1111'),
    'Subj_Crsnumb'
] = 'MATH 1111_0999'

merged_df.loc[
    (merged_df['HAS_SUPPORT'] == 1) & (merged_df['Subj_Crsnumb'] == 'MATH 1001'),
    'Subj_Crsnumb'
] = 'MATH 1001_0997'

merged_df.loc[
    (merged_df['HAS_SUPPORT'] == 1) & (merged_df['Subj_Crsnumb'] == 'MATH 1401'),
    'Subj_Crsnumb'
] = 'MATH 1401_0996'


In [146]:
# Drop standalone support course sections listings after they've been linked
support_only_courses = ['MATH 0999', 'MATH 0997', 'MATH 0996']
merged_df = merged_df[~merged_df['Subj_Crsnumb'].isin(support_only_courses)]

### Extract D2L Usernames from emails

> #### alovelace2@example.edu -> alovelace2



In [147]:
merged_df['Username'] = merged_df['Email'].str.split('@').str[0]

<ipython-input-147-beaf7e85e0b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Username'] = merged_df['Email'].str.split('@').str[0]


### Assign D2L role for course templates: *Template User or TA Designer*

In [148]:
# Define courses that require 'Template User' role
template_courses = ['MATH 1111', 'MATH 1111_0999', 'MATH 1401', 'MATH 1401_0996']

# Assign roles based on course
merged_df['Role'] = merged_df['Subj_Crsnumb'].apply(
    lambda x: 'Template User' if x in template_courses else 'TA Designer'
)


<ipython-input-148-7a8bae8ca2f0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Role'] = merged_df['Subj_Crsnumb'].apply(


### Prepare df to start creating Excel Report and D2L Faculty Roster files for upload

In [149]:
keep_columns = ['Subj_Crsnumb', 'CAMP','Campus_Name', 'INSTRUCTOR','FCST_CODE',
                'Email', 'HAS_SUPPORT', 'Username', 'Role']

In [150]:
# Define your preferred course display order
course_order = [
    'MATH 1001_0997', 'MATH 1001',
    'MATH 1111_0999', 'MATH 1111',
    'MATH 1401_0996', 'MATH 1401',
    'MATH 1113', 'MATH 2211', 'MATH 2212', 'MATH 2215',
    'MATH 2008', 'MATH 2420', 'MATH 2551', 'MATH 2641', 'MATH 2652'
]

# Apply this forced order to the Subj_Crsnumb column
merged_df['Subj_Crsnumb'] = pd.Categorical(
    merged_df['Subj_Crsnumb'],
    categories=course_order,
    ordered=True
)
merged_df = merged_df.sort_values('Subj_Crsnumb')

<ipython-input-150-cc9f7ef53325>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Subj_Crsnumb'] = pd.Categorical(


In [151]:
prepped_df = merged_df[keep_columns]

## 📊 Step 5: Create Excel Report

#### Create filename for report with timestamp

In [152]:
eastern = timezone('US/Eastern')
dateTimeObj = datetime.now(eastern)
datestamp = dateTimeObj.strftime("%Y_%b_%d") #_%I_%M%p") hours and minutes
report_name = "List_of_Instructors_by_Course_" + datestamp + ".xlsx"
print(report_name)

List_of_Instructors_by_Course_2025_May_20.xlsx


In [153]:
# Create writer object
writer = pd.ExcelWriter(report_name, engine='xlsxwriter')

In [154]:
# book object to format
workbook = writer.book

### List of Instructors by Campus

In [155]:
start_col = 0
start_row = 0
max_length = 0
sheetname = 'By Campus'
col_widths = {}

for campus in campuses:
    # Filter for current campus
    filtered = merged_df[merged_df['CAMP'] == campus]

    # Select relevant columns and sort
    temp_list = (
        filtered[['Campus_Name', 'INSTRUCTOR', 'FCST_CODE', 'Email']]
        .drop_duplicates()
        .sort_values('FCST_CODE')
    )

    if len(temp_list) > max_length:
        max_length = len(temp_list)

    # Write to Excel
    temp_list.to_excel(
        excel_writer=writer,
        sheet_name=sheetname,
        startrow=start_row,
        startcol=start_col,
        index=False
    )

    # Track max column widths across all blocks
    for idx, col in enumerate(temp_list.columns):
        col_values = temp_list[col].astype(str).tolist() + [col]
        max_len = max(map(len, col_values))
        col_idx = start_col + idx
        if col_idx not in col_widths or max_len > col_widths[col_idx]:
            col_widths[col_idx] = max_len

    # Advance to the next row block
    start_row += max_length + 3

    # If we overflow downward, shift to next column
    if start_row >= 30:
        start_row = 0
        max_length = 0
        start_col += len(temp_list.columns) + 1

# Apply final autosizing across all used columns
worksheet = writer.sheets[sheetname]
for col_idx, width in col_widths.items():
    worksheet.set_column(col_idx, col_idx, width + 2)


### Create List of instructors per course per campus

In [156]:
# Loop through each campus and subject to export instructor lists
for campus in campuses:
    start_col = 0
    start_row = 0
    max_length = 0
    col_widths = {}
    sheetname = campuses_dict[campus]

    # Filter by campus
    filtered = merged_df[merged_df['CAMP'] == campus]

    # Get list of unique subjects at this campus
    subjects_list = filtered['Subj_Crsnumb'].drop_duplicates().tolist()

    for subject in subjects_list:
        # Filter for this subject at this campus
        subject_df = filtered[filtered['Subj_Crsnumb'] == subject]

        # Select and deduplicate columns
        temp_list = subject_df[['Subj_Crsnumb', 'INSTRUCTOR', 'Email', 'Username']].drop_duplicates()

        if len(temp_list) > max_length:
            max_length = len(temp_list)

        # Write to Excel
        temp_list.to_excel(
            excel_writer=writer,
            sheet_name=sheetname,
            startrow=start_row,
            startcol=start_col,
            index=False
        )

        # Track max width for each column index across all blocks
        for idx, col in enumerate(temp_list.columns):
            col_values = temp_list[col].astype(str).tolist() + [col]
            max_len = max(map(len, col_values))
            col_idx = start_col + idx
            if col_idx not in col_widths or max_len > col_widths[col_idx]:
                col_widths[col_idx] = max_len


        # Advance the column block
        start_col += len(temp_list.columns) + 1

        # Line wrap if needed
        if start_col >= 15:
            start_row += max_length + 3
            start_col = 0
            max_length = 0

    # Auto-size columns AFTER all blocks are written
    worksheet = writer.sheets[sheetname]
    for col_idx, width in col_widths.items():
        worksheet.set_column(col_idx, col_idx, width + 2)

### Create Excel Sheets of Instructors by course (College-wide)

In [157]:
subjects_list = merged_df['Subj_Crsnumb'].drop_duplicates().tolist()
print(subjects_list)

for subject in subjects_list:
    sheetname = subject

    # Filter by subject
    filtered_df = merged_df[merged_df['Subj_Crsnumb'] == subject]

    # Select relevant columns and remove duplicates
    temp_list = filtered_df[['Subj_Crsnumb', 'INSTRUCTOR', 'Email', 'Username']].drop_duplicates()

    # Write to Excel
    temp_list.to_excel(
        excel_writer=writer,
        sheet_name=sheetname,
        index=False
    )

    # Auto-size columns
    worksheet = writer.sheets[sheetname]
    for idx, col in enumerate(temp_list.columns):
        column_len = max(
            temp_list[col].astype(str).map(len).max(),
            len(col)
        ) + 2
        worksheet.set_column(idx, idx, column_len)


['MATH 1001_0997', 'MATH 1001', 'MATH 1111_0999', 'MATH 1111', 'MATH 1401_0996', 'MATH 1401', 'MATH 1113', 'MATH 2211', 'MATH 2212', 'MATH 2215', 'MATH 2008', 'MATH 2420', 'MATH 2551', 'MATH 2641', 'MATH 2652']


In [158]:
writer.close()

##  📨 Step 6: Create course roster txt files of instructors for D2L upload into course templates

In [159]:
subjects_list = merged_df['Subj_Crsnumb'].drop_duplicates().tolist()

for subject in subjects_list:
    # Filter rows for this subject (Staff already removed)
    filtered = merged_df[merged_df['Subj_Crsnumb'] == subject]

    # Get unique Username and Role pairs
    temp_list = filtered[['Username', 'Role']].drop_duplicates()

    # Create "Username, Role" strings
    roster_list = temp_list.apply(lambda row: f"{row['Username']}, {row['Role']}", axis=1).tolist()

    # Write to file
    import_roster = f"{subject} D2L instructor roster.txt"
    with open(import_roster, 'w') as f:
        f.write("\n".join(roster_list))